In [3]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import re


# Initialize the web driver (Chrome WebDriver should be in your PATH)
driver = webdriver.Chrome()

# Website URL
url = 'https://photofinish.live/marketplace/a99022fe-2255-454a-a6e9-0f3375cb20c0'  # Your specific URL here

try:
    # Navigate to the URL
    driver.get(url)

    # Wait for the page to load (you may need to adjust the wait time)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@class="flex flex-row gap-2"]')))

    # Get the page source after navigating to the specific link
    page_source = driver.page_source

    # Parse the page source with BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')

    # Scrape the desired information
    horse_name = soup.find('div', class_='flex flex-row gap-2').find('span', class_='text-2xl').text.strip() if soup.find('div', class_='flex flex-row gap-2') else None
    age = soup.find('div', class_='flex flex-row max-w-full whitespace-nowrap sm:text-xs').find('span').text.strip() if soup.find('div', class_='flex flex-row max-w-full whitespace-nowrap sm:text-xs') else None
    fleet_figure = soup.find('span', class_='text-xs').text.strip() if soup.find('span', class_='text-xs') else None
    # gender = soup.find('div', class_='flex flex-row justify-start items-center text-sky-300').text.strip() if soup.find('div', class_='flex flex-row justify-start items-center text-sky-300') else None
    stable_name = soup.find('a', class_='text-secondary-100/80 hover:text-secondary-400 underline block pl-1 cursor-pointer').text.strip() if soup.find('a', class_='text-secondary-100/80 hover:text-secondary-400 underline block pl-1 cursor-pointer') else None
    
    # Check for gender in the first location
    gender_element = soup.find('div', class_='flex flex-row justify-start items-center text-sky-300')
    if gender_element:
        gender = gender_element.text.strip()
    else:
    # If not found in the first location, check the second location
        gender_element = soup.find('div', class_='flex flex-row justify-start items-center text-pink-300')
        if gender_element:
            gender = gender_element.text.strip()
        else:
            gender = None
    
    # Extracting the price using a regular expression
    price_pattern = r'Purchase (\d+,\d+\.\d+)'
    price_match = re.search(price_pattern, page_source)
    if price_match:
        horse_price = price_match.group(1)
    else:
        horse_price = None

    # Extracting the horse's status
    status_elements = soup.find_all('div', class_='flex flex-row') + soup.find_all('p', class_='text-white ml-1')
    horse_status = None

    for element in status_elements:
        if 'Retired' in element.text:
            horse_status = 'Retired'
            break
        elif 'Exhausted' in element.text:
            horse_status = 'Exhausted'
            break
        elif 'Foal' in element.text:
            horse_status = 'Foal'
            break
        elif 'Ready to Race!' in element.text:
            horse_status = 'Ready to Race!'
        elif 'Pregnant' in element.text:
            horse_status = 'Pregnant'
            break

    # Extracting Starts(W-P-S)
    starts_element = soup.find('p', class_='text-sm text-white font-inter')
    starts_data = starts_element.text.strip() if starts_element else None

    # Example code for extracting career earnings
    career_earnings_element = soup.find_all('p', class_='text-sm text-slate-300')
    if len(career_earnings_element) >= 1:
        career_earnings = career_earnings_element[0].text.strip()
    else:
        career_earnings = None

    # Example code for extracting largest purse
    if len(career_earnings_element) >= 2:
        largest_purse = career_earnings_element[1].text.strip()
    else:
        largest_purse = None

    # Extracting Career Injuries
    injuries_element = soup.find('p', class_='font-inter text-sm text-white')
    career_injuries = injuries_element.text.strip() if injuries_element else None

    # Print the scraped information
    print("Horse Name:", horse_name)
    print("Age:", age)
    print("Fleet Figure:", fleet_figure)
    print("Gender:", gender)
    print("Stable Name:", stable_name)
    print("Horse Price:", horse_price)
    print("Horse Status:", horse_status)
    print("Starts(W-P-S):", starts_data)
    print("Career Earnings:", career_earnings)
    print("Largest Purse:", largest_purse)
    print("Career Injuries:", career_injuries)

    # Save the scraped data to a DataFrame
    scraped_data = pd.DataFrame({
        'Horse Name': [horse_name],
        'Age': [age],
        'Fleet Figure': [fleet_figure],
        'Gender': [gender],
        'Stable Name': [stable_name],
        'Horse Price': [horse_price],
        'Horse Status': [horse_status],
        'Starts(W-P-S)': [starts_data],
        'Career Earnings': [career_earnings],
        'Largest Purse': [largest_purse],
        'Career Injuries': [career_injuries]
    })

    # Assign a unique Horse ID
    scraped_data['Horse ID'] = 1  # You can assign a different unique ID if needed

    # Save the scraped data to a CSV file (customize the filename and path as needed)
    scraped_data.to_csv('scraped_data.csv', index=False)

    try:
        # Scroll to the CSV download button
        csv_download_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//a[contains(text(), "Download CSV")]'))
        )
        csv_download_button.click()

        # Wait for the download to complete (you may need to adjust the wait time)
        time.sleep(10)  # Adjust as needed

    except Exception as e:
        print("CSV not available.")

except Exception as e:
    print("An error occurred:", str(e))

finally:
    # Close the browser window when done
    driver.quit()


Horse Name: Sol Snatcher
Age: Age 6
Fleet Figure: 47
Gender: Mare
Stable Name: Shooting Stars
Horse Price: 82,000.00
Horse Status: Ready to Race!
Starts(W-P-S): 36 (4-7-3)
Career Earnings: 29,086.62
Largest Purse: 9,024.00
Career Injuries: 0


## Combine Both The CSV Files

In [26]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Initialize the web driver (Chrome WebDriver should be in your PATH)
driver = webdriver.Chrome()

# Website URL
url = 'https://photofinish.live/marketplace/a99022fe-2255-454a-a6e9-0f3375cb20c0'  # Your specific URL here

try:
    # Navigate to the URL
    driver.get(url)

    # Wait for the page to load (you may need to adjust the wait time)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@class="flex flex-row gap-2"]')))

    # Get the page source after navigating to the specific link
    page_source = driver.page_source

    # Parse the page source with BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')

    # Scrape the desired information (same code as before)
    horse_name = soup.find('div', class_='flex flex-row gap-2').find('span', class_='text-2xl').text.strip() if soup.find('div', class_='flex flex-row gap-2') else None
    age = soup.find('div', class_='flex flex-row max-w-full whitespace-nowrap sm:text-xs').find('span').text.strip() if soup.find('div', class_='flex flex-row max-w-full whitespace-nowrap sm:text-xs') else None
    fleet_figure = soup.find('span', class_='text-xs').text.strip() if soup.find('span', class_='text-xs') else None
    # gender = soup.find('div', class_='flex flex-row justify-start items-center text-sky-300').text.strip() if soup.find('div', class_='flex flex-row justify-start items-center text-sky-300') else None
    stable_name = soup.find('a', class_='text-secondary-100/80 hover:text-secondary-400 underline block pl-1 cursor-pointer').text.strip() if soup.find('a', class_='text-secondary-100/80 hover:text-secondary-400 underline block pl-1 cursor-pointer') else None
    
    # Check for gender in the first location
    gender_element = soup.find('div', class_='flex flex-row justify-start items-center text-sky-300')
    if gender_element:
        gender = gender_element.text.strip()
    else:
    # If not found in the first location, check the second location
        gender_element = soup.find('div', class_='flex flex-row justify-start items-center text-pink-300')
        if gender_element:
            gender = gender_element.text.strip()
        else:
            gender = None
    
    # Extracting the price using a regular expression
    price_pattern = r'Purchase (\d+,\d+\.\d+)'
    price_match = re.search(price_pattern, page_source)
    if price_match:
        horse_price = price_match.group(1)
    else:
        horse_price = None

    # Extracting the horse's status
    status_elements = soup.find_all('div', class_='flex flex-row') + soup.find_all('p', class_='text-white ml-1')
    horse_status = None

    for element in status_elements:
        if 'Retired' in element.text:
            horse_status = 'Retired'
            break
        elif 'Exhausted' in element.text:
            horse_status = 'Exhausted'
            break
        elif 'Foal' in element.text:
            horse_status = 'Foal'
            break
        elif 'Ready to Race!' in element.text:
            horse_status = 'Ready to Race!'
        elif 'Pregnant' in element.text:
            horse_status = 'Pregnant'
            break

    # Extracting Starts(W-P-S)
    starts_element = soup.find('p', class_='text-sm text-white font-inter')
    starts_data = starts_element.text.strip() if starts_element else None

    # Example code for extracting career earnings
    career_earnings_element = soup.find_all('p', class_='text-sm text-slate-300')
    if len(career_earnings_element) >= 1:
        career_earnings = career_earnings_element[0].text.strip()
    else:
        career_earnings = None

    # Example code for extracting largest purse
    if len(career_earnings_element) >= 2:
        largest_purse = career_earnings_element[1].text.strip()
    else:
        largest_purse = None

    # Extracting Career Injuries
    injuries_element = soup.find('p', class_='font-inter text-sm text-white')
    career_injuries = injuries_element.text.strip() if injuries_element else None

    # Print the scraped information
    print("Horse Name:", horse_name)
    print("Age:", age)
    print("Fleet Figure:", fleet_figure)
    print("Gender:", gender)
    print("Stable Name:", stable_name)
    print("Horse Price:", horse_price)
    print("Horse Status:", horse_status)
    print("Starts(W-P-S):", starts_data)
    print("Career Earnings:", career_earnings)
    print("Largest Purse:", largest_purse)
    print("Career Injuries:", career_injuries)

    # Save the scraped data to a DataFrame
    scraped_data = pd.DataFrame({
        'Horse Name': [horse_name],
        'Age': [age],
        'Fleet Figure': [fleet_figure],
        'Gender': [gender],
        'Stable Name': [stable_name],
        'Horse Price': [horse_price],
        'Horse Status': [horse_status],
        'Starts(W-P-S)': [starts_data],
        'Career Earnings': [career_earnings],
        'Largest Purse': [largest_purse],
        'Career Injuries': [career_injuries]
    })

    # Assign a unique Horse ID
    scraped_data['Horse ID'] = 1  # You can assign a different unique ID if needed

    # Save the scraped data to a CSV file (customize the filename and path as needed)
    scraped_data.to_csv('scraped_data.csv', index=False)
    
    # Scroll to the CSV download button
    csv_download_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//a[contains(text(), "Download CSV")]'))
    )
    csv_download_button.click()

    # Wait for the download to complete (you may need to adjust the wait time)
    time.sleep(10)  # Adjust as needed

    # Load the scraped horse data
    scraped_horse_data = pd.read_csv('scraped_data.csv')

    # Load the downloaded race data
    downloaded_race_data = pd.read_csv(r'C:\Users\kathan\Downloads\completed-races.csv')

    # Repeat scraped horse data for each row of race data
    repeated_horse_data = pd.concat([scraped_horse_data] * len(downloaded_race_data), ignore_index=True)

    # Combine the two DataFrames column-wise
    combined_data = pd.concat([downloaded_race_data, repeated_horse_data], axis=1)

    # Save the combined data to a new CSV file
    combined_data.to_csv('combined_data.csv', index=False)

except Exception as e:
    print("An error occurred:", str(e))

finally:
    # Close the browser window when done
    driver.quit()

Horse Name: Sol Snatcher
Age: Age 6
Fleet Figure: 47
Gender: Mare
Stable Name: Shooting Stars
Horse Price: 82,000.00
Horse Status: Ready to Race!
Starts(W-P-S): 36 (4-7-3)
Career Earnings: 29,086.62
Largest Purse: 9,024.00
Career Injuries: 0


## Change File Location Code

In [27]:
import os
import shutil

# Set the path to your default download directory
default_download_dir = r'C:\Users\kathan\Downloads'  # Replace YourUsername with your actual username

# Set the path to your desired directory
desired_directory = r'E:\Web Scraping\Login Jupyter\temp'

# Specify the file name you want to move
file_to_move = 'completed-races.csv'

# Check if the file exists in the default download directory
if os.path.exists(os.path.join(default_download_dir, file_to_move)):
    # Create the full source path and destination path
    source_path = os.path.join(default_download_dir, file_to_move)
    destination_path = os.path.join(desired_directory, file_to_move)

    # Move the file from the default download directory to the desired directory
    try:
        shutil.move(source_path, destination_path)
        print(f"File '{file_to_move}' moved successfully to '{desired_directory}'.")
    except Exception as e:
        print(f"Error moving the file: {e}")
else:
    print(f"File '{file_to_move}' not found in the default download directory.")


File 'completed-races.csv' moved successfully to 'E:\Web Scraping\Login Jupyter\temp'.


## Combination Of Both Functionalities

In [28]:
import os
import shutil
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Set the path to your default download directory
default_download_dir = r'C:\Users\kathan\Downloads'  # Replace with your actual username

# Set the path to your desired directory
desired_directory = r'E:\Web Scraping\Login Jupyter\temp'

# Specify the file name you want to move and combine
file_to_move = 'completed-races.csv'

# Initialize the web driver (Chrome WebDriver should be in your PATH)
driver = webdriver.Chrome()

# Website URL
url = 'https://photofinish.live/marketplace/a99022fe-2255-454a-a6e9-0f3375cb20c0'  # Your specific URL here

try:
    # Navigate to the URL
    driver.get(url)

    # Wait for the page to load (you may need to adjust the wait time)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@class="flex flex-row gap-2"]')))

    # Get the page source after navigating to the specific link
    page_source = driver.page_source
    
     # Parse the page source with BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')

    # Scrape the desired information (same code as before)
    horse_name = soup.find('div', class_='flex flex-row gap-2').find('span', class_='text-2xl').text.strip() if soup.find('div', class_='flex flex-row gap-2') else None
    age = soup.find('div', class_='flex flex-row max-w-full whitespace-nowrap sm:text-xs').find('span').text.strip() if soup.find('div', class_='flex flex-row max-w-full whitespace-nowrap sm:text-xs') else None
    fleet_figure = soup.find('span', class_='text-xs').text.strip() if soup.find('span', class_='text-xs') else None
    # gender = soup.find('div', class_='flex flex-row justify-start items-center text-sky-300').text.strip() if soup.find('div', class_='flex flex-row justify-start items-center text-sky-300') else None
    stable_name = soup.find('a', class_='text-secondary-100/80 hover:text-secondary-400 underline block pl-1 cursor-pointer').text.strip() if soup.find('a', class_='text-secondary-100/80 hover:text-secondary-400 underline block pl-1 cursor-pointer') else None
    
    # Check for gender in the first location
    gender_element = soup.find('div', class_='flex flex-row justify-start items-center text-sky-300')
    if gender_element:
        gender = gender_element.text.strip()
    else:
    # If not found in the first location, check the second location
        gender_element = soup.find('div', class_='flex flex-row justify-start items-center text-pink-300')
        if gender_element:
            gender = gender_element.text.strip()
        else:
            gender = None
    
    # Extracting the price using a regular expression
    price_pattern = r'Purchase (\d+,\d+\.\d+)'
    price_match = re.search(price_pattern, page_source)
    if price_match:
        horse_price = price_match.group(1)
    else:
        horse_price = None

    # Extracting the horse's status
    status_elements = soup.find_all('div', class_='flex flex-row') + soup.find_all('p', class_='text-white ml-1')
    horse_status = None

    for element in status_elements:
        if 'Retired' in element.text:
            horse_status = 'Retired'
            break
        elif 'Exhausted' in element.text:
            horse_status = 'Exhausted'
            break
        elif 'Foal' in element.text:
            horse_status = 'Foal'
            break
        elif 'Ready to Race!' in element.text:
            horse_status = 'Ready to Race!'
        elif 'Pregnant' in element.text:
            horse_status = 'Pregnant'
            break

    # Extracting Starts(W-P-S)
    starts_element = soup.find('p', class_='text-sm text-white font-inter')
    starts_data = starts_element.text.strip() if starts_element else None

    # Example code for extracting career earnings
    career_earnings_element = soup.find_all('p', class_='text-sm text-slate-300')
    if len(career_earnings_element) >= 1:
        career_earnings = career_earnings_element[0].text.strip()
    else:
        career_earnings = None

    # Example code for extracting largest purse
    if len(career_earnings_element) >= 2:
        largest_purse = career_earnings_element[1].text.strip()
    else:
        largest_purse = None

    # Extracting Career Injuries
    injuries_element = soup.find('p', class_='font-inter text-sm text-white')
    career_injuries = injuries_element.text.strip() if injuries_element else None

    # Print the scraped information
    print("Horse Name:", horse_name)
    print("Age:", age)
    print("Fleet Figure:", fleet_figure)
    print("Gender:", gender)
    print("Stable Name:", stable_name)
    print("Horse Price:", horse_price)
    print("Horse Status:", horse_status)
    print("Starts(W-P-S):", starts_data)
    print("Career Earnings:", career_earnings)
    print("Largest Purse:", largest_purse)
    print("Career Injuries:", career_injuries)

    # Save the scraped data to a DataFrame
    scraped_data = pd.DataFrame({
        'Horse Name': [horse_name],
        'Age': [age],
        'Fleet Figure': [fleet_figure],
        'Gender': [gender],
        'Stable Name': [stable_name],
        'Horse Price': [horse_price],
        'Horse Status': [horse_status],
        'Starts(W-P-S)': [starts_data],
        'Career Earnings': [career_earnings],
        'Largest Purse': [largest_purse],
        'Career Injuries': [career_injuries]
    })

    # Assign a unique Horse ID
    scraped_data['Horse ID'] = 1  # You can assign a different unique ID if needed

    # Save the scraped data to a CSV file (customize the filename and path as needed)
    scraped_data.to_csv('scraped_data.csv', index=False)
    
    # Scroll to the CSV download button
    csv_download_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, '//a[contains(text(), "Download CSV")]'))
    )
    csv_download_button.click()

    # Wait for the download to complete (you may need to adjust the wait time)
    time.sleep(10)  # Adjust as needed
    
    # Move the downloaded file to the desired directory
    if os.path.exists(os.path.join(default_download_dir, file_to_move)):
        source_path = os.path.join(default_download_dir, file_to_move)
        destination_path = os.path.join(desired_directory, file_to_move)

        try:
            shutil.move(source_path, destination_path)
            print(f"File '{file_to_move}' moved successfully to '{desired_directory}'.")
        except Exception as e:
            print(f"Error moving the file: {e}")
    else:
        print(f"File '{file_to_move}' not found in the default download directory.")

    # Load the downloaded race data from the temp directory
    downloaded_race_data = pd.read_csv(os.path.join(desired_directory, file_to_move))

    # Load the scraped horse data (modify the path as needed)
    scraped_horse_data = pd.read_csv('scraped_data.csv')

    # Repeat scraped horse data for each row of race data
    repeated_horse_data = pd.concat([scraped_horse_data] * len(downloaded_race_data), ignore_index=True)

    # Combine the two DataFrames column-wise
    combined_data = pd.concat([downloaded_race_data, repeated_horse_data], axis=1)

    # Set the path to the final directory
    final_directory = r'E:\Web Scraping\Login Jupyter\final'

    # Save the combined data to the final directory
    combined_data.to_csv(os.path.join(final_directory, 'combined_data.csv'), index=False)
    print(f"Combined data saved to '{final_directory}'.")

except Exception as e:
    print("An error occurred:", str(e))

finally:
    # Close the browser window when done
    driver.quit()

Horse Name: Sol Snatcher
Age: Age 6
Fleet Figure: 47
Gender: Mare
Stable Name: Shooting Stars
Horse Price: 82,000.00
Horse Status: Ready to Race!
Starts(W-P-S): 36 (4-7-3)
Career Earnings: 29,086.62
Largest Purse: 9,024.00
Career Injuries: 0
File 'completed-races.csv' moved successfully to 'E:\Web Scraping\Login Jupyter\temp'.
Combined data saved to 'E:\Web Scraping\Login Jupyter\final'.


## Multiple Horse Links 

In [30]:
import os
import shutil
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup

# Set the path to your default download directory
default_download_dir = r'C:\Users\kathan\Downloads'  # Replace with your actual username

# Set the path to your desired directory
desired_directory = r'E:\Web Scraping\Login Jupyter\temp'

# Initialize the web driver (Chrome WebDriver should be in your PATH)
driver = webdriver.Chrome()

# Website URLs for horse links
horse_links = [
    'https://photofinish.live/marketplace/a99022fe-2255-454a-a6e9-0f3375cb20c0',
    'https://photofinish.live/marketplace/76a6937d-cebe-4466-a3e9-41f86d8a07cc',
    'https://photofinish.live/marketplace/a998e4bb-580d-4904-8523-6050b68da8bd'
    # Add more horse links as needed
]

try:
    # Initialize a unique horse ID counter
    horse_id_counter = 1

    for horse_link in horse_links:
        # Navigate to the horse link
        driver.get(horse_link)

        # Wait for the page to load (you may need to adjust the wait time)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@class="flex flex-row gap-2"]')))

        # Get the page source after navigating to the specific link
        page_source = driver.page_source

        # Parse the page source with BeautifulSoup
        soup = BeautifulSoup(page_source, 'html.parser')

        # Scrape the desired information
        horse_name = soup.find('div', class_='flex flex-row gap-2').find('span', class_='text-2xl').text.strip() if soup.find('div', class_='flex flex-row gap-2') else None
        age = soup.find('div', class_='flex flex-row max-w-full whitespace-nowrap sm:text-xs').find('span').text.strip() if soup.find('div', class_='flex flex-row max-w-full whitespace-nowrap sm:text-xs') else None
        fleet_figure = soup.find('span', class_='text-xs').text.strip() if soup.find('span', class_='text-xs') else None
        stable_name = soup.find('a', class_='text-secondary-100/80 hover:text-secondary-400 underline block pl-1 cursor-pointer').text.strip() if soup.find('a', class_='text-secondary-100/80 hover:text-secondary-400 underline block pl-1 cursor-pointer') else None
        
        gender_element = soup.find('div', class_='flex flex-row justify-start items-center text-sky-300')
        if gender_element:
            gender = gender_element.text.strip()
        else:
            gender_element = soup.find('div', class_='flex flex-row justify-start items-center text-pink-300')
            if gender_element:
                gender = gender_element.text.strip()
            else:
                gender = None
        
        price_pattern = r'Purchase (\d+,\d+\.\d+)'
        price_match = re.search(price_pattern, page_source)
        if price_match:
            horse_price = price_match.group(1)
        else:
            horse_price = None

        status_elements = soup.find_all('div', class_='flex flex-row') + soup.find_all('p', class_='text-white ml-1')
        horse_status = None

        for element in status_elements:
            if 'Retired' in element.text:
                horse_status = 'Retired'
                break
            elif 'Exhausted' in element.text:
                horse_status = 'Exhausted'
                break
            elif 'Foal' in element.text:
                horse_status = 'Foal'
                break
            elif 'Ready to Race!' in element.text:
                horse_status = 'Ready to Race!'
            elif 'Pregnant' in element.text:
                horse_status = 'Pregnant'
                break

        starts_element = soup.find('p', class_='text-sm text-white font-inter')
        starts_data = starts_element.text.strip() if starts_element else None

        career_earnings_element = soup.find_all('p', class_='text-sm text-slate-300')
        if len(career_earnings_element) >= 1:
            career_earnings = career_earnings_element[0].text.strip()
        else:
            career_earnings = None

        if len(career_earnings_element) >= 2:
            largest_purse = career_earnings_element[1].text.strip()
        else:
            largest_purse = None

        injuries_element = soup.find('p', class_='font-inter text-sm text-white')
        career_injuries = injuries_element.text.strip() if injuries_element else None

        # Print the scraped information
        print("Horse Name:", horse_name)
        print("Age:", age)
        print("Fleet Figure:", fleet_figure)
        print("Gender:", gender)
        print("Stable Name:", stable_name)
        print("Horse Price:", horse_price)
        print("Horse Status:", horse_status)
        print("Starts(W-P-S):", starts_data)
        print("Career Earnings:", career_earnings)
        print("Largest Purse:", largest_purse)
        print("Career Injuries:", career_injuries)

        try:
            # Scroll to the CSV download button
            csv_download_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//a[contains(text(), "Download CSV")]'))
            )
            csv_download_button.click()

            # Wait for the download to complete (you may need to adjust the wait time)
            time.sleep(10)  # Adjust as needed

            # Move the downloaded file to the desired directory
            if os.path.exists(os.path.join(default_download_dir, file_to_move)):
                source_path = os.path.join(default_download_dir, file_to_move)
                destination_path = os.path.join(desired_directory, file_to_move)

                try:
                    shutil.move(source_path, destination_path)
                    print(f"File '{file_to_move}' moved successfully to '{desired_directory}'.")
                except Exception as e:
                    print(f"Error moving the file: {e}")
            else:
                print(f"File '{file_to_move}' not found in the default download directory.")

        except Exception as e:
            print(f"CSV not available for {horse_link}. Adding scraped data directly to combined data CSV...")

            # Assign a unique Horse ID
            horse_id = horse_id_counter
            horse_id_counter += 1

            # Create a DataFrame for the scraped horse data
            scraped_data = pd.DataFrame({
                'Horse ID': [horse_id],
                'Horse Name': [horse_name],
                'Age': [age],
                'Fleet Figure': [fleet_figure],
                'Gender': [gender],
                'Stable Name': [stable_name],
                'Horse Price': [horse_price],
                'Horse Status': [horse_status],
                'Starts(W-P-S)': [starts_data],
                'Career Earnings': [career_earnings],
                'Largest Purse': [largest_purse],
                'Career Injuries': [career_injuries]
            })

            # Save the scraped data to a CSV file (customize the filename and path as needed)
            scraped_data.to_csv(os.path.join(desired_directory, f'scraped_data_{horse_id}.csv'), index=False)

except Exception as e:
    print("An error occurred:", str(e))

finally:
    # Close the browser window when done
    driver.quit()

Horse Name: Sol Snatcher
Age: Age 6
Fleet Figure: 47
Gender: Mare
Stable Name: Shooting Stars
Horse Price: 82,000.00
Horse Status: Ready to Race!
Starts(W-P-S): 36 (4-7-3)
Career Earnings: 29,086.62
Largest Purse: 9,024.00
Career Injuries: 0
File 'completed-races.csv' moved successfully to 'E:\Web Scraping\Login Jupyter\temp'.
Horse Name: Slow Clap
Age: Age 2
Fleet Figure: 40
Gender: Colt
Stable Name: Alen ibro
Horse Price: 28,950.00
Horse Status: Ready to Race!
Starts(W-P-S): 16 (1-3-0)
Career Earnings: 2,893.68
Largest Purse: 1,373.76
Career Injuries: 0
File 'completed-races.csv' moved successfully to 'E:\Web Scraping\Login Jupyter\temp'.
Horse Name: Foal of Online Oathkeeper
Age: Foal
Fleet Figure: 00
Gender: Filly
Stable Name: RVA Racing
Horse Price: 65,000.00
Horse Status: Foal
Starts(W-P-S): 0 (0-0-0)
Career Earnings: 0.00
Largest Purse: 0.00
Career Injuries: None
CSV not available for https://photofinish.live/marketplace/a998e4bb-580d-4904-8523-6050b68da8bd. Adding scraped data 

In [54]:
import os
import shutil
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
import re

# Set the path to your default download directory
default_download_dir = r'C:\Users\kathan\Downloads'  # Replace with your actual username

# Set the path to your desired directory
desired_directory = r'E:\Web Scraping\Login Jupyter\temp'

# Initialize the web driver (Chrome WebDriver should be in your PATH)
driver = webdriver.Chrome()

# Website URLs for horse links
horse_links = [
    'https://photofinish.live/marketplace/a99022fe-2255-454a-a6e9-0f3375cb20c0',
    'https://photofinish.live/marketplace/047ba1e1-860c-47d8-ab29-e2ea1882242a',
    'https://photofinish.live/marketplace/a998e4bb-580d-4904-8523-6050b68da8bd'
    # Add more horse links as needed
]

try:
    # Initialize a unique horse ID counter
    horse_id_counter = 1

    for horse_link in horse_links:
        # Navigate to the horse link
        driver.get(horse_link)

        # Wait for the page to load (you may need to adjust the wait time)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@class="flex flex-row gap-2"]')))

        # Get the page source after navigating to the specific link
        page_source = driver.page_source

        # Check if CSV is available, if not, skip this URL
        if 'CSV not available' in page_source:
            print("CSV not available for", horse_link)
            continue

        # Parse the page source with BeautifulSoup
        soup = BeautifulSoup(page_source, 'html.parser')

        # Scrape the desired information (same code as before)
        horse_name = soup.find('div', class_='flex flex-row gap-2').find('span', class_='text-2xl').text.strip() if soup.find('div', class_='flex flex-row gap-2') else None
        age = soup.find('div', class_='flex flex-row max-w-full whitespace-nowrap sm:text-xs').find('span').text.strip() if soup.find('div', class_='flex flex-row max-w-full whitespace-nowrap sm:text-xs') else None
        fleet_figure = soup.find('span', class_='text-xs').text.strip() if soup.find('span', class_='text-xs') else None
        stable_name = soup.find('a', class_='text-secondary-100/80 hover:text-secondary-400 underline block pl-1 cursor-pointer').text.strip() if soup.find('a', class_='text-secondary-100/80 hover:text-secondary-400 underline block pl-1 cursor-pointer') else None
        
        gender_element = soup.find('div', class_='flex flex-row justify-start items-center text-sky-300')
        if gender_element:
            gender = gender_element.text.strip()
        else:
            gender_element = soup.find('div', class_='flex flex-row justify-start items-center text-pink-300')
            if gender_element:
                gender = gender_element.text.strip()
            else:
                gender = None
        
        price_pattern = r'Purchase (\d+,\d+\.\d+)'
        price_match = re.search(price_pattern, page_source)
        if price_match:
            horse_price = price_match.group(1)
        else:
            horse_price = None

        status_elements = soup.find_all('div', class_='flex flex-row') + soup.find_all('p', class_='text-white ml-1')
        horse_status = None

        for element in status_elements:
            if 'Retired' in element.text:
                horse_status = 'Retired'
                break
            elif 'Exhausted' in element.text:
                horse_status = 'Exhausted'
                break
            elif 'Foal' in element.text:
                horse_status = 'Foal'
                break
            elif 'Ready to Race!' in element.text:
                horse_status = 'Ready to Race!'
            elif 'Pregnant' in element.text:
                horse_status = 'Pregnant'
                break

        starts_element = soup.find('p', class_='text-sm text-white font-inter')
        starts_data = starts_element.text.strip() if starts_element else None

        career_earnings_element = soup.find_all('p', class_='text-sm text-slate-300')
        if len(career_earnings_element) >= 1:
            career_earnings = career_earnings_element[0].text.strip()
        else:
            career_earnings = None

        if len(career_earnings_element) >= 2:
            largest_purse = career_earnings_element[1].text.strip()
        else:
            largest_purse = None

        injuries_element = soup.find('p', class_='font-inter text-sm text-white')
        career_injuries = injuries_element.text.strip() if injuries_element else None

        # Print the scraped information
        print("Horse Name:", horse_name)
        print("Age:", age)
        print("Fleet Figure:", fleet_figure)
        print("Gender:", gender)
        print("Stable Name:", stable_name)
        print("Horse Price:", horse_price)
        print("Horse Status:", horse_status)
        print("Starts(W-P-S):", starts_data)
        print("Career Earnings:", career_earnings)
        print("Largest Purse:", largest_purse)
        print("Career Injuries:", career_injuries)
        

        # Save the scraped data to a DataFrame
        scraped_data = pd.DataFrame({
            'Horse Name': [horse_name],
            'Age': [age],
            'Fleet Figure': [fleet_figure],
            'Gender': [gender],
            'Stable Name': [stable_name],
            'Horse Price': [horse_price],
            'Horse Status': [horse_status],
            'Starts(W-P-S)': [starts_data],
            'Career Earnings': [career_earnings],
            'Largest Purse': [largest_purse],
            'Career Injuries': [career_injuries]
        })

        # Assign a unique Horse ID
        scraped_data['Horse ID'] = horse_id_counter
        horse_id_counter += 1

        # Save the scraped data to a CSV file (customize the filename and path as needed)
        scraped_data.to_csv(f'scraped_data_{horse_id_counter}.csv', index=False)

        # Scroll to the CSV download button
        csv_download_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//a[contains(text(), "Download CSV")]'))
        )
        csv_download_button.click()

        # Wait for the download to complete (you may need to adjust the wait time)
        time.sleep(10)  # Adjust as needed

        # Move the downloaded file to the desired directory
        if os.path.exists(os.path.join(default_download_dir, file_to_move)):
            source_path = os.path.join(default_download_dir, file_to_move)
            destination_path = os.path.join(desired_directory, file_to_move)

            try:
                shutil.move(source_path, destination_path)
                print(f"File '{file_to_move}' moved successfully to '{desired_directory}'.")
            except Exception as e:
                print(f"Error moving the file: {e}")
        else:
            print(f"File '{file_to_move}' not found in the default download directory.")

        # Load the downloaded race data from the temp directory
        downloaded_race_data = pd.read_csv(os.path.join(desired_directory, file_to_move))

        # Load the scraped horse data (modify the path as needed)
        scraped_horse_data = pd.read_csv(f'scraped_data_{horse_id_counter}.csv')

        # Repeat scraped horse data for each row of race data
        repeated_horse_data = pd.concat([scraped_horse_data] * len(downloaded_race_data), ignore_index=True)

        # Combine the two DataFrames column-wise
        combined_data = pd.concat([downloaded_race_data, repeated_horse_data], axis=1)

        # Set the path to the final directory
        final_directory = r'E:\Web Scraping\Login Jupyter\final'

        # Save the combined data to the final directory
        combined_data.to_csv(os.path.join(final_directory, f'combined_data_{horse_id_counter}.csv'), index=False)
        print(f"Combined data saved to '{final_directory}'.")

except Exception as e:
    print("An error occurred:", str(e))

finally:
    # Close the browser window when done
    driver.quit()

Horse Name: Sol Snatcher
Age: Age 6
Fleet Figure: 47
Gender: Mare
Stable Name: Shooting Stars
Horse Price: 82,000.00
Horse Status: Ready to Race!
Starts(W-P-S): 36 (4-7-3)
Career Earnings: 29,086.62
Largest Purse: 9,024.00
Career Injuries: 0
File 'completed-races.csv' moved successfully to 'E:\Web Scraping\Login Jupyter\temp'.
Combined data saved to 'E:\Web Scraping\Login Jupyter\final'.
Horse Name: Byte Baron
Age: Age 6
Fleet Figure: 59
Gender: Stallion
Stable Name: OnlyFoalsAndHorses
Horse Price: 9,200.00
Horse Status: Ready to Race!
Starts(W-P-S): 15 (1-2-0)
Career Earnings: 1,692.00
Largest Purse: 648.00
Career Injuries: 0
File 'completed-races.csv' moved successfully to 'E:\Web Scraping\Login Jupyter\temp'.
Combined data saved to 'E:\Web Scraping\Login Jupyter\final'.
Horse Name: Foal of Online Oathkeeper
Age: Foal
Fleet Figure: 00
Gender: Filly
Stable Name: RVA Racing
Horse Price: 65,000.00
Horse Status: Foal
Starts(W-P-S): 0 (0-0-0)
Career Earnings: 0.00
Largest Purse: 0.00
Care